rpc permissions?


In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var listInProject = importer.import('list project files');
var FUNCTION_GROUPS = ['Function', 'Public'];
var SELENIUM_GROUPS = ['Selenium'];
var UNITTEST_GROUPS = ['Unit Test']; // TODO: identified using describe
var DEFAULT_GROUPS = []
var PUBLIC = {
    'google contacts.ipynb[google contact settings]': ['Function', 'Inner Circle', 'Public'],
    'import notebook.ipynb[search notebook questions]' : ['Function', 'Public'],
    'rpc.ipynb[permissions]' : ['Function', 'Public'],
    'rpc.ipynb[parameter names]' : ['Function', 'Public'],
    'gulp.ipynb[search notebooks]' : ['Function', 'Public'],
    'data collection.ipynb[meta search all]' : ['Function', 'Public'],
    'data collection.ipynb[schedule search all]' : ['Function', 'Public'],
    'data collection.ipynb[tell joke]' : ['Function', 'Public'],
};

var REGEX_EXPORTS = /exports[^=]*?\s*=\s*([^ ]*)\s*;/igm;
var REGEX_NAMED = /function\s+([^\( ]*)\s*\(/igm

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function getExports(code) {
    const named = regexToArray(REGEX_NAMED, code, 1);
    const namedExports = regexToArray(REGEX_EXPORTS, code, 1);
    return _.intersection(named, namedExports);
}

// TODO: move these classifications to import notebook?
var AVAILABLE;
function restorePermissions() {
    const cacheFilename = path.resolve(path.join(__dirname, './cached-permissions.json'));
    if(typeof __webpack_require__ === 'function'
          && typeof getCachedPermissions !== 'undefined') {
        AVAILABLE = getCachedPermissions();
        return;
    } else if (fs.existsSync(cacheFilename)) {
        const ptime = fs.statSync(cacheFilename).mtime.getTime();
        const ntime = listInProject('..', '{,*,*/,*/*/*,*/*/*/*,*/*/*/*/*}*.ipynb').map(a => fs.statSync(a).mtime.getTime()).sort().pop();
        if(ptime > ntime) {
            AVAILABLE = JSON.parse(fs.readFileSync(cacheFilename).toString());
            return;
        }
    }
    var defined = importer.interpret(Object.keys(PUBLIC)).map(r => r.id);
    var empty = importer.interpret().filter(r => defined.indexOf(r.id) === -1);
    AVAILABLE = empty.reduce((acc, r) => {
        const id = r.id.replace(/\[.*]/ig, '') + '[' + r.question + ']';
        // TODO: find the shortest words from the query to match the sam cell
        // TODO: filter RPC functions by module.exports = <name> and function <name> matching function name?
        try {
            const search = importer.interpret(id);
            if(search.id !== r.id) {
                console.error('Not going to work ' + id);
            } 
            if(search.filename.indexOf('Selenium') > -1) {
                if(getExports(search.code).length > 0) {
                    acc[id] = SELENIUM_GROUPS.concat(FUNCTION_GROUPS);
                } else {
                    acc[id] = SELENIUM_GROUPS;
                }
                // TODO: called using selenium runner?
            } else if(getExports(search.code).length > 0) {
                acc[id] = FUNCTION_GROUPS;
            } else {
                acc[id] = [];
            }
        } catch (e) { // ignore
        }
        return acc;
    }, {});
    fs.writeFileSync(cacheFilename, JSON.stringify(Object.assign({}, PUBLIC, AVAILABLE), null, 4));
}

// TODO: filter RPC functions by fully unit tested?
// TODO: filter by local system groups?

function getPermissions() {
    if(typeof AVAILABLE === 'undefined') {
        restorePermissions();
    }
    // Always return the permission set so it can't be manipulated by another command.
    return Object.assign({}, PUBLIC, AVAILABLE);
}

if(typeof $$ !== 'undefined') {
    console.log(getPermissions());
}
module.exports = getPermissions;


filter command permission?


In [ ]:
var importer = require('../Core');
var getPermissions = importer.import('rpc permissions');
var getDaysEvents = importer.import('days events');
var getSettings = importer.import('google contact settings');
var getOauthClient = importer.import('import google calendar api');

var options = {
    calendarId: 'Commands'
};

// TODO: move this logic out to a higher level coordinator?
function filterCommand(command, date, id, user) {
    const props = {};
    return getOauthClient(options)
        .then(() => getSettings(user))
        .then(settings => {
            // assign user controls and interpreted command
            Object.assign(props, settings);
            try {
                console.log(settings.circles)
                Object.assign(props, {
                    result: importer.interpret(command)
                });
                // if looking for settings use the user name provided to this function for lookup
                // TODO: accept parameters from message context
                if(props.result.id === importer.interpret('google contacts.ipynb[google contact settings]').id) {
                    props.result.runInNewContext = () => settings;
                }
            } catch (e) {
                if((e + '').indexOf('Nothing found') > -1) {
                    Object.assign(props, {result: null});
                } else {
                    throw e;
                }
            }
        })
        .then(() => getDaysEvents(new Date(date), options))
        .then(events => {
            // filter already sent
            const already = events
              .filter(e => e.event.summary.indexOf('Result:') > -1 && e.event.summary.indexOf(id) > -1).length > 0;
            Object.assign(props, {already: already})
        })
        .catch(e => console.log(e))
        .then(() => props)
}
module.exports = filterCommand;


store rpc result?


In [ ]:
var importer = require('../Core');
var updateEvent = importer.import('update create merge event');
var ISODateString = importer.import('convert date iso');
var getOauthClient = importer.import('import google calendar api');
var getParameterNames = importer.import('function parameters');
var getResult = importer.import('rpc result');

var options = {
    calendarId: 'Commands'
};

function updateResultEvent(response, friend, isError, isStarting = false) {
    const config = {
        start: {
            dateTime: ISODateString(new Date(friend.date.getTime()))
        },
        end: {
            dateTime: ISODateString(new Date(friend.date.getTime() + 60 * 30 * 1000))
        },
        summary: 'Result: ' + friend.id,
        description: JSON.stringify(response, null, 4),
        colorId: isStarting ? 9 : (isError ? 11 : 10)
    }
    return updateEvent(config, options).then(() => response);
}

function storeResult(friend) {
    if(typeof friend === 'undefined' || friend === null
       || friend.already !== false) {
        // skip commands that have already been run
        throw new Error('Nothing to do!');
    }
    
    var isError;
    return getOauthClient(options)
        // create a new events to store the results
        // process the command, this should return a function to be called after event
        .then(() => updateResultEvent('starting ' + friend.result.filename, friend, false, true))
        .then(() => getResult(friend))
        .catch(e => {
            console.log(e);
            isError = true;
            return Object.getOwnPropertyNames(e).reduce((alt, key) => {
                alt[key] = e[key];
                return alt;
            }, {})
        })
        // store as an event for tracking
        .then(response => updateResultEvent(response, friend, isError))
        .catch(e => console.log(e))
}
module.exports = storeResult;



rpc result?


In [ ]:
var path = require('path');
var importer = require('../Core');
var getPermissions = importer.import('rpc permissions');
var getParameterNames = importer.import('function parameters',
                                        // prevent recursion
                                        {getResult}); 

// apply parameters is set to false if the caller wants a validated function returned
function getResult(props, applyParameters = true) {
    // filter permissions
    var result;
    var isEmail = false;
    if(typeof props.subject !== 'undefined') {
        isEmail = true;
    }
    
    if (props.result === null) {
        throw new Error('Command not found, please specify a command '
                         + (isEmail ? 'in the subject line' : '')
                         + ' like "megamind settings" without quotes.');
    }
    
    // compare id with permissions
    const permissions = getPermissions();
    Object.assign(props, {
        allowed: props.circles
            .filter(c => {
                const matching = Object.keys(permissions)
                    .filter(k => permissions[k].indexOf(c) > -1 || permissions[k].indexOf('Public') > -1);
                return matching.filter(k => path.basename(props.result.filename) === k.split('[')[0]
                                       && importer.interpret(k).id === props.result.id).length > 0;
            }).length > 0
    });

    // pass error up
    if(!props.allowed) {
        throw new Error('Would have run "' + props.result.id + '" but you don\'t have permission. Deploy your own server to get access to all RPC functions.');
    }
    
    // TODO: make this nicer, ugly because importer doesn't conform to the same importing
    //   style and therefore functions are missing from the context when loaded separately.
    // This is maybe a sign there is something wrong with this style of dependency injection
    var commandResult = props.result.runInNewContext(
        props.result.id.indexOf('rpc.ipynb') === 0
        ? {getResult}
        : (props.result.id.indexOf('import notebook.ipynb') === 0
           ? importer
           : {}));
    if(typeof commandResult === 'object' && commandResult !== null
        && typeof commandResult[Object.keys(commandResult)[0]] === 'function') {
        commandResult = commandResult[Object.keys(commandResult)[0]];
    }
    if(applyParameters && typeof commandResult === 'function') {
        const parameters = getParameterNames(commandResult);
        const parameterValues = props.body
            ? parameters.slice(1).map((k, i) => {
                const p = props.body[k] || props.body[i];
                if(typeof p === 'undefined' || p === 'undefined') {
                    return;
                }
                return p;
            })
            : [];
        return commandResult.apply(importer, parameterValues);
    } else {
        return commandResult;
    }
}
module.exports = getResult;



get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');

var COMMENTS = /((\/\/.*$)|(\/\*[\s\S]*?\*\/))/mg;
var DEFAULT_PARAMS = /=[^,]+/mg;
var FAT_ARROWS = /=>[\s\S]*$|{[\s\S]*$/mg;

function getParameterNames(search) {
    var result;
    if(typeof search === 'function') {
        result = search;
    } else {
        if(typeof getResult === 'undefined') {
            var getResult = importer.import('rpc result');
        }
        result = getResult({
            command: search,
            result: importer.interpret(search),
            circles: ['Function']
        }, false);
    }

    var fnName = result.name;
    
    var code = result.toString()
        .replace(COMMENTS, '')
        .replace(FAT_ARROWS, '')
        .replace(DEFAULT_PARAMS, '');

    var result = code.slice(code.indexOf('(') + 1, code.indexOf(')'))
        .match(/([^\s,]+)/g);

    return result === null
        ? [fnName]
        : [fnName, ...result];
}

module.exports = getParameterNames;
